## Pré-processamento do DataFrame com músicas

In [270]:
from lingua import Language, LanguageDetectorBuilder
import pandas as pd
import re

In [258]:
df = pd.read_csv('musicas_artistas.csv')
df.drop(['indice_artista', 'indice_musica', 'identificador', 'tags'], axis=1, inplace=True)

In [259]:
df.sample(10)

,titulo_musica,nome_artista,letra,album,data_lancamento
3768,Ando Meio Desligado,BRAZA,"[Letra de ""Ando Meio Desligado"" com BRAZA]\n\n...",NaN,"Oct. 9, 2020"
2060,Ai que Bom,Tim Maia,NaN,Sufocante (1984),1 Viewer
12426,Jaguar,Bemti,[Intro]\n*coro cantando em outra linguagem*\n\...,era dois (2018),"Aug. 3, 2018"
11059,Has Logrado Olvidar,Miami Sound Machine,Cuando emociones olvidadas vuelves a sentir\nL...,Otra Vez (1981),1981
1347,Do Nada,Belo,"[Letra de ""Do Nada"" com Belo]\n\nDo nada\nEntr...",De Alma Aberta (2018),"Apr. 17, 2018"
15635,Lata d’água na Cabeça,As Baías,"Leva lata, lata d'água na cabeça\nLeva lata, l...",Mulher (2015),"Nov. 6, 2015"
15572,Enquanto Você Não Chega,Illy (BRA),"Letra de ""Enquanto Você Não Chega"" de Illy\nEn...",Voo Longe (2018),"Apr. 12, 2018"
12006,The Jolly Corner,Rogério Skylab,"Esperei, procurei, eu só queria achar\nMas eu ...",Os Cosmonautas (2020),"Dec. 24, 2020"
10657,Hoy (Album Version),Gloria Estefan,Tengo marcado en el pecho\nTodos los días que ...,NaN,"Jul. 29, 2003"
9032,Apocalipsom A (O fim no palco no começo),Tom Zé,Personae iures alieni\nDiabo e Deus numa sala\...,Tropicália Lixo Lógico (2012),"Jan. 1, 2012"


In [260]:
df.shape

(17526, 5)

#### Algumas observações:
1. Há algumas músicas que não estão em português
2. Há algumas músicas MPB sem a tag de MPB
3. Alguns artistas cantam músicas iguais

#### Pré-processamento:

1. Criar uma coluna chamada 'ano_lancamento' que captura o ano da coluna 'album' ou da coluna 'data_lancamento'
2. Remover linhas com letras duplicadas (mantendo a primeira) e com letras nulas ou vazias
3. Remover partes das letras entre colchetes, substituir quebra de linha por espaço, deixar letras minúsculas
4. Filtrar músicas em português

### 1. Criando a coluna com ano de lançamento

In [261]:
def extrair_ano_album(texto):
    if pd.notna(texto):
        padrao_ano = re.compile(r'\((\d{4})\)')
        resultado = padrao_ano.search(str(texto))
        return resultado.group(1) if resultado else None
    return None

def extrair_ano_data(texto):
    if pd.notna(texto):
        padrao_ano = re.compile(r'(\d{4})')
        resultado = padrao_ano.search(str(texto))
        return resultado.group(1) if resultado else None
    return None

In [262]:
def extrair_ano(linha):
    ano_album = extrair_ano_album(linha['album'])
    ano_data = extrair_ano_data(linha['data_lancamento'])
    return ano_album or ano_data

df['ano_lancamento'] = df.apply(extrair_ano, axis=1)
df.drop(['album', 'data_lancamento'], axis=1, inplace=True)
df.drop(df[df['ano_lancamento'] == '0001'].index, inplace=True)
df = df.dropna(subset=['ano_lancamento'])
df = df.sort_values(by='ano_lancamento')

In [263]:
df.head(5)

,titulo_musica,nome_artista,letra,ano_lancamento
14748,Pablo Neruda - Poema 20 (English Translation),Genius English Translations,[translated by W.S. Merwin]\n\nTonight I can w...,1924
14955,Pablo Neruda - Every Day You Play (English Tra...,Genius English Translations,Every day you play with the light of the unive...,1924
14864,Herms Niel - Erika (English Translation),Genius English Translations,"[Verse 1]\nOn the\n heath, there blooms a litt...",1930
14286,Vasco Scansani di Gualtieri - Bella Ciao (Engl...,Genius English Translations,"One morning I awakened\nOh Goodbye beautiful, ...",1943
14640,Édith Piaf - La vie en rose (English Translation),Genius English Translations,[Verse 1]\nEyes that lower mine\nA laugh which...,1945


In [264]:
df.tail()

,titulo_musica,nome_artista,letra,ano_lancamento
14463,Jin - The Astronaut (English Translation),Genius English Translations,"[Verse 1]\nYou and me, an unending history\nOh...",2024
8576,Tenho Que Ser*,PRISCILLA,"Letra da Prévia\n\nNão quero aparecer, eu só t...",2024
1348,Grip,Anitta,Lyrics from live performance\n\n[Intro]\nQue h...,2024
8618,Falso Bom Rapaz,PRISCILLA,NaN,2024
8505,Nem Aí,PEDRO SAMPAIO & MC Ryan SP,"[Letra de ""Nem Aí"" com PEDRO SAMPAIO & MC Ryan...",2024


### 2. Removendo linhas com letras duplicadas e com letras nulas (mantendo a primeira)

In [265]:
df = df.dropna(subset=['letra'])
df = df.drop_duplicates(subset = 'letra', keep = 'first')
df.shape

(15846, 4)

In [266]:
df.sample(5)

,titulo_musica,nome_artista,letra,ano_lancamento
2480,Ninguém Pode Mais Sofrer,Geraldo Vandré,Quem vai me escutar?\nQuem vai me entender?\nN...,1964
11443,Ela Diz Que Me Ama,Roberta Sá,"[Letra de ""Ela Diz Que Me Ama"" com Roberta Sá]...",2019
6565,Rosto Barbado,Marcos Valle,Você já tentou acordar mais cedo do que faz?\n...,1972
13220,Não Vou Te Deixar,Gaby Amarantos,"[Letra de ""Não Vou Te Deixar"" com Gaby Amarant...",2022
2364,Quem Me Conhece Sabe,Ana Vilela,"[Letra de ""Quem Me Conhece Sabe"" com Ana Vilel...",2023


### 3. Removendo partes das letras entre colchetes, substituindo quebra de linha por espaço e deixando letras minúsculas

In [267]:
def remover_partes_entre_colchetes(texto):
    return re.sub(r'\[.*?\]', '', texto)

df['letra'] = df['letra'].apply(remover_partes_entre_colchetes)
df['letra'] = df['letra'].replace('\n', ' ', regex=True)
df['letra'] = df['letra'].apply(lambda x: x.lower())

In [268]:
df.sample(10)

,titulo_musica,nome_artista,letra,ano_lancamento
16398,Terminei Indo,China (BRA),eu já sei caminhar em tantas nuvens e posso vi...,2011
10469,MAMACITA (Fodassy remix),"Luísa Sonza, Xamã & Karol Conká","yeah, ya, ya, ya yeah, ya, ya, ya mamacita fa...",2023
1001,Januária,Vinícius de Moraes,"- olá chiquinho, essa é pra você - oi chico. ...",1975
11000,Dr. Beat (Extended Version),Miami Sound Machine,emergency! paging dr. beat! emergency! doctor...,1984
17439,"Alegria, Minha Alegria!",Martinho da Vila,"eu tenho uma alegria particular fofinha, sens...",2016
12195,Rosa Roseira,Ivete Sangalo,eu tiro uma rosa da roseira e levo pra te pres...,2000
4729,Las Muchachas de la Plaza España,Roberto Carlos,es la vieja roma en primavera una enredadera d...,1997
4532,Ele Está Pra Chegar,Roberto Carlos,se ilumine na luz das estrelas se aqueça nos r...,1981
6568,These Are the Songs,Elis Regina,these are the songs i want to sing these are t...,1970
16701,Angu Com Caroço,Almir Sater,a coisa tá feia o caldo tá grosso o povo na ru...,2022


### 4. Filtrando músicas em português

In [305]:
def detecta_idioma(texto):
    languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.JAPANESE, Language.PORTUGUESE]
    detector = LanguageDetectorBuilder.from_languages(*languages).build()
    language = detector.detect_language_of(texto)
    
    return str(language)

In [306]:
df['idioma'] = df['letra'].apply(detecta_idioma)

In [307]:
df['idioma'].value_counts()

Language.PORTUGUESE    13428
Language.ENGLISH        2203
Language.FRENCH          164
None                      33
Language.GERMAN           17
Language.JAPANESE          1
Name: idioma, dtype: int64

In [309]:
df[df['idioma'] == 'None']

,titulo_musica,nome_artista,letra,ano_lancamento,idioma
5529,Deixa,Toquinho,,1966,None
178,Catavento,Milton Nascimento,,1967,None
816,Tema para Morte e Vida Severina,Chico Buarque,,1968,None
6705,Verão Vermelho,Elis Regina,,1970,None
11532,Sarro,Gilberto Gil & Jorge Ben,,1975,None
6844,Osher - אושר,Matti Caspi - מתי כספי,העצב אין לו סוף לאושר יש ויש אושר הוא נוצה קלה...,1978,None
1053,Keilu Sof - כאילו סוף,Hava Alberstein - חוה אלברשטיין,כאילו כאן סוף סוף נרגעת נדידה הנח ראשך המיוגע ...,1983,None
923,Ho Ma Yihiye? - ?הו מה יהיה,Matti Caspi - מתי כספי,הוי מה יהיה מה יהיה תגידי מה יהיה כאן איתי ואי...,1987,None
10048,Noturna,Elba Ramalho,,1995,None
7035,Vehageshem Yavo - והגשם יבוא,Gidi Gov - גידי גוב,"מקל ואבן והשביל שאבד זה סוף המסלול, זו טיפונת...",1997,None
